In [51]:
import pandas as pd

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
#查看缺失值
#print(train_data.info())
#用中值填充缺失值
#train_data["Age"] = train_data["Age"].fillna(train_data["Age"].median())
print(train_data.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


In [52]:
print(train_data["Sex"].unique())
#对sex进行数值化处理
train_data.loc[train_data["Sex"] == 'male', "Sex"] = 0
train_data.loc[train_data["Sex"] == 'female', "Sex"] = 1

['male' 'female']


In [53]:
#将Parch、SibSp合并
train_data["FamilySize"] = train_data["Parch"] + train_data["SibSp"]
train_data["FamilySize"][train_data["FamilySize"] == 0] = 0
train_data["FamilySize"][train_data["FamilySize"] == 1] = 1
print(train_data)

     PassengerId  Survived  Pclass  ... Cabin Embarked  FamilySize
0              1         0       3  ...   NaN        S           1
1              2         1       1  ...   C85        C           1
2              3         1       3  ...   NaN        S           0
3              4         1       1  ...  C123        S           1
4              5         0       3  ...   NaN        S           0
..           ...       ...     ...  ...   ...      ...         ...
886          887         0       2  ...   NaN        S           0
887          888         1       1  ...   B42        S           0
888          889         0       3  ...   NaN        S           3
889          890         1       1  ...  C148        C           0
890          891         0       3  ...   NaN        Q           0

[891 rows x 13 columns]


<ipython-input-53-b622dbdf4919>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data["FamilySize"][train_data["FamilySize"] == 0] = 0
<ipython-input-53-b622dbdf4919>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data["FamilySize"][train_data["FamilySize"] == 1] = 1


In [54]:
train_data["Fare"][train_data["Fare"] < 20] = 1
train_data["Fare"][train_data["Fare"] >= 20] = 0
train_data.head()

<ipython-input-54-ce115d77687c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data["Fare"][train_data["Fare"] < 20] = 1
<ipython-input-54-ce115d77687c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data["Fare"][train_data["Fare"] >= 20] = 0


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,1.0,NaN,S,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,0.0,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,1.0,NaN,S,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,0.0,C123,S,1
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,1.0,NaN,S,0


In [50]:
#删掉没用的colnum，用随机森林预测缺失的值
#print(train_data)
#train_data.drop(['PassengerId', 'Name', 'SibSp', 'Parch','Ticket', 'Cabin','Embarked'], inplace=True, axis = 1)
train_data.loc[(train_data["Age"] < 18 & train_data["Age"].notnull())] = 0
train_data.loc[(train_data["Age"] >= 18 & train_data["Age"].notnull())] = 1
y_train = train_data["Age"][train_data.Age.notnull()].values#将值取出来
print(train_data.info())
print(y_train.shape)
x_train = train_data[train_data.Age.notnull()].drop(['Age'],axis=1).values#删除Age为空的行
print(x_train.shape)
x_test = train_data[train_data.Age.isnull()].drop(['Age'],axis=1).values
print(x_test.shape)

from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier().fit(x_train, y_train)
train_data["Age"][train_data["Age"].isnull()] = rfc.predict(x_test)
train_data.info()

train_data = pd.DataFrame(train_data)
train_data.head()

     Survived  Pclass Sex  Age  Fare  FamilySize
0           1       1   1  1.0   1.0           1
1           1       1   1  1.0   1.0           1
2           1       1   1  1.0   1.0           1
3           1       1   1  1.0   1.0           1
4           1       1   1  1.0   1.0           1
..        ...     ...  ..  ...   ...         ...
886         1       1   1  1.0   1.0           1
887         1       1   1  1.0   1.0           1
888         1       1   1  1.0   1.0           1
889         1       1   1  1.0   1.0           1
890         1       1   1  1.0   1.0           1

[891 rows x 6 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Survived    891 non-null    int64  
 1   Pclass      891 non-null    int64  
 2   Sex         891 non-null    object 
 3   Age         891 non-null    float64
 4   Fare        891 non-null    float

ValueError: Found array with 0 sample(s) (shape=(0, 5)) while a minimum of 1 is required.